### Prelims

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

### Model Architecture

- 전체 모델의 구조를 정의하는 기본 클래스이다.
- 주요 컴포넌트 : encoder, decoder, src_embed(입력 시퀀스를 임베딩하는 함수), tgt_embed(출력 시퀀스를 임베딩하는 함수), generator(디코더의 출력을 최종적으로 변환하여 출력 확률을 생헌하는 컴포넌트)로 구성이 되어있다.
- 주요 메서드 :
    - forward() - 마스크가 적용된 입출력 시퀀스를 받아 인코더와 디코더를 차례로 호출한다.
    - encode() - 입력 데이터를 임베딩한 후 인코더를 통해 처리한다.
    - decode() - 인코더의 출력 결과(memory)를 받아 디코더에서 처리한다.

In [2]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)
        "입력 시퀀스 src를 인코더에 전달해 memory(연속 표현) 생성"
        "이후 memory와 디코더에 입력 시퀀스 tgt를 전달해 출력 생성"
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

- 디코더의 출력 결과를 실제 어휘 확률 분포로 변환
- 주요 컴퍼넌트
    
    proj : dmodel → vocab size로 변환하는 선형 계층
    
- 주요 메서드
    
    forward(x) : 디코더 출력 x에 대해 선형 변환을 수행한 후 log-softmax를 적용하여 확률 분포 반환

In [5]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

### Encoder와 Decoder 구조 상세 설명

##### 🚀 Encoder 🚀
- 인코더는 6개의 동일한 레이어(stack)으로 구성
- 레이어 구성 = 2개의 서브레이어(sub-layer)으로 이루어져있으며, residual connection과 layer normalization이 적용됨

In [6]:
"Encoder class"
class Encoder(nn.Module) :
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N) #N개의 동일한 레이어 복제
        self.norm = LayerNorm(layer.size) #마지막 출력 정규화

    def forward(self, x, mask):
        for layer in self.layers: #레이어를 순차적으로 통과
            x = layer(x, mask)
        return self.norm(x) # 정규화 후 최종 출력
    
"Layernorm class"
class LayerNorm(nn.Module) :
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features)) # 가중치(초깃값 : 1)
        self.b_2 = nn.Parameter(torch.zeros(features)) # 편향(초깃값 : 0)
        self.eps = eps
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

- 서브레이어 : Residual Connection

In [7]:
class SublayerConnection(nn.Module):
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)  # 정규화
        self.dropout = nn.Dropout(dropout)  # 드롭아웃

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))  # 잔차 연결

- Encoder Layer 클래스  
2개의 서브 레이어로 구성 : Multi-head Self-Attention & Feed-forward Network

In [8]:
class EncoderLayer(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn  # Self-Attention
        self.feed_forward = feed_forward  # Feed-Forward Network
        self.sublayer = clones(SublayerConnection(size, dropout), 2)  # 2개의 서브레이어
        self.size = size

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))  # Self-Attention
        return self.sublayer[1](x, self.feed_forward)  # Feed-Forward


##### 🚀 Decoder 🚀  
- 디코더도 6개의 동일한 레이어로 구성이 되며, 3개의 서브레이어를 포함한다
    1. Multi-head Self-Attention
    2. Source-Target Attention (인코더의 출력과 디코더 입력 간의 상호작용)
    3. Feed-forward Network

In [9]:
class Decoder(nn.Module):
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)
    
class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)  # 3개의 서브레이어

    def forward(self, x, memory, src_mask, tgt_mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))  # Self-Attention
        x = self.sublayer[1](x, lambda x: self.src_attn(x, memory, memory, src_mask))  # Source-Target Attention
        return self.sublayer[2](x, self.feed_forward)  # Feed-Forward

def subsequent_mask(size):
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0
